In [8]:
from datasets import load_dataset

ds = load_dataset("mary-ruiliii/Genshin-character-portrait-image-character-data-all", split="train")
print(ds[0])
print(type(ds[0]["image"]))

ds = ds.filter(lambda x: x["name"] != "Wanderer")


{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=476x476 at 0x7FB051697280>, 'text': 'A human Genshin-style character. uses a sword. has a Geo vision. medium male body figure. from Mondstadt.', 'name': 'Albedo', 'region': 'Mondstadt', 'vision': 'Geo', 'weapon_type': 'Sword', 'body_figure': 'Medium Male', 'rarity': 5, 'constellation': 'Princeps Cretaceus', 'affiliation': 'Knights of Favonius'}
<class 'PIL.PngImagePlugin.PngImageFile'>


Filter: 100%|██████████| 73/73 [00:00<00:00, 240.68 examples/s]


In [11]:
print(len(ds))

72


In [2]:
def add_names(example, idx):
    if "name" not in example:
        example["name"] = ""                     # <-- important: always set something
    return example

ds = ds.map(add_names, with_indices=True)
ds = ds.filter(lambda x: x["name"] != "")

for i in range(3):
    print(ds[i])


{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=643x643 at 0x7FB1E646EFE0>, 'text': 'A human Genshin-style character. uses a claymore. has a Hydro vision. short female body figure. from Nod-Krai.', 'name': 'Aino'}
{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=476x476 at 0x7FB1E641B730>, 'text': 'A human Genshin-style character. uses a sword. has a Geo vision. medium male body figure. from Mondstadt.', 'name': 'Albedo'}
{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=536x536 at 0x7FB052F7F730>, 'text': 'A human Genshin-style character. uses a sword. has a Dendro vision. tall male body figure. from Sumeru.', 'name': 'Alhaitham'}


In [123]:
ori_images = ds["image"]

In [3]:
import pandas as pd
df = pd.read_csv("filtered_output.csv").map(lambda x: x["affiliation"])

print(df.head())

  character_name     region  vision weapon_type    body_figure  rarity  \
0         Albedo  Mondstadt     Geo       Sword    Medium Male       5   
1      Alhaitham     Sumeru  Dendro       Sword      Tall Male       5   
2           Aloy        NaN    Cryo         Bow  Medium Female       5   
3          Amber  Mondstadt    Pyro         Bow  Medium Female       4   
4   Arataki Itto    Inazuma     Geo    Claymore      Tall Male       5   

        constellation          affiliation  
0  Princeps Cretaceus  Knights of Favonius  
1       Vultur Volans     Sumeru Akademiya  
2         Nora Fortis    Wandering Heroine  
3               Lepus  Knights of Favonius  
4    Taurus Iracundus         Arataki Gang  


In [4]:
hf_df = ds.to_pandas()

merged = hf_df.merge(df, left_on="name", right_on="character_name", how="inner")

# drop character_name
merged = merged.drop("character_name", axis = 1)
print(merged.head())

                                               image  \
0  {'bytes': None, 'path': '/home/ubuntu/.cache/h...   
1  {'bytes': None, 'path': '/home/ubuntu/.cache/h...   
2  {'bytes': None, 'path': '/home/ubuntu/.cache/h...   
3  {'bytes': None, 'path': '/home/ubuntu/.cache/h...   
4  {'bytes': None, 'path': '/home/ubuntu/.cache/h...   

                                                text          name     region  \
0  A human Genshin-style character. uses a sword....        Albedo  Mondstadt   
1  A human Genshin-style character. uses a sword....     Alhaitham     Sumeru   
2  A human Genshin-style character. uses a bow. h...         Amber  Mondstadt   
3  A human Genshin-style character. uses a claymo...  Arataki Itto    Inazuma   
4  A human Genshin-style character. uses a cataly...        Baizhu      Liyue   

   vision weapon_type    body_figure  rarity       constellation  \
0     Geo       Sword    Medium Male       5  Princeps Cretaceus   
1  Dendro       Sword      Tall Male    

In [5]:
from datasets import Dataset
ds_final = Dataset.from_pandas(merged, preserve_index=False)
print(ds_final)

Dataset({
    features: ['image', 'text', 'name', 'region', 'vision', 'weapon_type', 'body_figure', 'rarity', 'constellation', 'affiliation'],
    num_rows: 73
})


In [6]:
from datasets import Dataset, Features, Image, Value

features = Features({
    "image": Image(),          # must be Image() for PIL images
    "text": Value("string"),
    "name": Value("string"),
    "region": Value("string"),
    "vision": Value("string"),
    "weapon_type": Value("string"),
    "body_figure": Value("string"),
    "rarity": Value("int32"),
    "constellation": Value("string"),
    "affiliation": Value("string"),
})

In [128]:
merged = merged.drop(columns=["image"])  # remove the bad path-based image column
ds_final = Dataset.from_pandas(merged, features=features, preserve_index=False)
ds_final = ds_final.map(
    lambda ex, idx: {"image": ori_images[idx]},
    with_indices=True
)
ds_final

Map:  88%|████████▊ | 64/73 [00:00<00:00, 199.51 examples/s]

Map: 100%|██████████| 73/73 [00:00<00:00, 195.22 examples/s]


Dataset({
    features: ['image', 'text', 'name', 'region', 'vision', 'weapon_type', 'body_figure', 'rarity', 'constellation', 'affiliation'],
    num_rows: 73
})

In [ ]:
from huggingface_hub import HfApi

# api = HfApi()
# api.create_repo("mary-ruiliii/Genshin-character-portrait-image-character-data-three", repo_type="dataset")
ds.push_to_hub("mary-ruiliii/Genshin-character-portrait-image-character-data-all")

Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 43.65ba/s]
Processing Files (1 / 1): 100%|██████████| 13.7MB / 13.7MB,  0.00B/s  
New Data Upload: |          |  0.00B /  0.00B,  0.00B/s  
Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.19s/ shards]
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/datasets/mary-ruiliii/Genshin-character-portrait-image-character-data-all/commit/b937854931e944d5b7e2c22e09de51716c62105b', commit_message='Upload dataset', commit_description='', oid='b937854931e944d5b7e2c22e09de51716c62105b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mary-ruiliii/Genshin-character-portrait-image-character-data-all', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mary-ruiliii/Genshin-character-portrait-image-character-data-all'), pr_revision=None, pr_num=None)

In [130]:
ds_final[0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=643x643>,
 'text': 'A human Genshin-style character. uses a sword. has a Geo vision. medium male body figure. from Mondstadt.',
 'name': 'Albedo',
 'region': 'Mondstadt',
 'vision': 'Geo',
 'weapon_type': 'Sword',
 'body_figure': 'Medium Male',
 'rarity': 5,
 'constellation': 'Princeps Cretaceus',
 'affiliation': 'Knights of Favonius'}